### Importar librerias

In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import spacy
import re 

import warnings
warnings.filterwarnings("ignore")
import os
os.chdir(r'C:\Users\sngh9\OneDrive\Escritorio\Maestria_Semestre_2\NLP')
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier

### Lectura de datos

In [2]:
datasetMaterial = pd.read_json(r'jsonDatosAjustado.json', 'utf-8')
#datasetMaterial = pd.read_json('/content/drive/MyDrive/Proyecto1PLN/ProyectoPLN.ipynb', encoding='latin-1')


#### Registros unicos

In [3]:
datasetMaterial['dropkey'] = datasetMaterial['text'] + datasetMaterial['key'] + datasetMaterial['url']

In [4]:
datasetMaterial= datasetMaterial.drop_duplicates(subset=['url'], keep='first').reset_index(drop=True)

In [5]:
datasetMaterial.shape

(47571, 10)

#### Imputacion de faltante 

In [6]:
datasetMaterial.text[datasetMaterial['text']=='']=datasetMaterial.title[datasetMaterial['text']=='']

#### Quitar accentos

In [7]:
import unicodedata
def strip_accents(s):
   # Retorna el string normalizado si la categoría es Mn es decir https://www.compart.com/en/unicode/category/Mn
   return ''.join(str(c).lower() for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')
datasetMaterial['text'] = datasetMaterial['text'].apply(strip_accents)

In [8]:
datasetMaterial.topic.value_counts(normalize=True)

deportes      0.311618
cultura       0.232768
vida          0.142839
politica      0.133043
economia      0.120872
tecnosfera    0.058859
Name: topic, dtype: float64

### NLP

In [9]:
nlpSpa = spacy.load('es_core_news_lg')

In [10]:
df=datasetMaterial.sample(frac=0.05)
df

,_id,topic,text,title,year,month,day,key,url,dropkey
33977,{'$oid': '611d9fe6049812a18b162994'},politica,el presidente ivan duque llamo ayer al expresi...,¿De qué hablaron Duque y Uribe en llamada tras...,2020,8,5,politica-gobierno-presidente-ivan-duque-se-pro...,http://www.eltiempo.com//politica/gobierno/pre...,El presidente Iván Duque llamó ayer al expresi...
10983,{'$oid': '611d9fe3049812a18b146d35'},cultura,"“no me afecta, la gente puede decir lo que qui...",Hijo de Gloria Trevi habla sobre los ataques a...,2021,4,30,cultura-musica-y-libros-hijo-de-gloria-trevi-s...,http://www.eltiempo.com//cultura/musica-y-libr...,"“No me afecta, la gente puede decir lo que qui..."
38588,{'$oid': '611d9fe7049812a18b16aaa9'},tecnosfera,el consumer electronic show (ces) de las vegas...,"Televisores inteligentes, 5G y las novedades d...",2020,1,8,tecnosfera-novedades-tecnologia-novedades-que-...,http://www.eltiempo.com//tecnosfera/novedades-...,El Consumer Electronic Show (CES) de Las Vegas...
43605,{'$oid': '611d9fe8049812a18b1787ba'},vida,en uno de los momentos de la historia en que m...,Qué dice la neurobiología de la distancia soci...,2020,12,12,vida-que-dice-la-neurobiologia-de-la-distancia...,http://www.eltiempo.com//vida/que-dice-la-neur...,En uno de los momentos de la historia en que m...
31021,{'$oid': '611d9fe5049812a18b15e51c'},economia,segun un nuevo estudio de oracle y workplace i...,La pandemia hizo del 2020 el año laboral más e...,2020,10,12,economia-sectores-coranavirus-el-2020-es-el-an...,http://www.eltiempo.com//economia/sectores/cor...,Según un nuevo estudio de Oracle y Workplace I...
...,...,...,...,...,...,...,...,...,...,...
4982,{'$oid': '611d9fe2049812a18b1403ac'},cultura,este 20 de agosto se conocio el particular vid...,Video: el curioso protocolo de bioseguridad de...,2020,8,21,cultura-gente-barranquilla-particular-protocol...,http://www.eltiempo.com//cultura/gente/barranq...,Este 20 de agosto se conoció el particular vid...
8912,{'$oid': '611d9fe2049812a18b144842'},cultura,pocos desafios mas complejos para un escritor ...,Así mataron a papá: Jorge Emilio Sierra,2020,3,28,cultura-musica-y-libros-asi-mataron-a-papa-fra...,http://www.eltiempo.com//cultura/musica-y-libr...,Pocos desafíos más complejos para un escritor ...
36116,{'$oid': '611d9fe6049812a18b165728'},politica,la modalidad de contrato por horas fue uno de ...,¿El Congreso abrió la puerta al contrato de tr...,2020,8,26,politica-congreso-proyecto-que-estableceria-el...,http://www.eltiempo.com//politica/congreso/pro...,La modalidad de contrato por horas fue uno de ...
40827,{'$oid': '611d9fe8049812a18b17327d'},vida,"'soy de la generacion igualdad’, dice el lema ...",El empoderamiento de la mujer cuestiona reinad...,2020,3,14,vida-mujeres-el-empoderamiento-de-la-mujer-cue...,http://www.eltiempo.com//vida/mujeres/el-empod...,"'Soy de la generación igualdad’, dice el lema ..."


In [11]:
df['text'] = df['text'].apply(nlpSpa)

In [12]:
def PosFeatures(doc):

    lst=[token.tag_ for token in doc]
    Adjs=lst.count('ADJ')
    Advs=lst.count('ADV')
    Aux=lst.count('AUX')
    Conj=lst.count('CONJ')
    Det=lst.count('DET')
    Inter=lst.count('INTJ')
    Nouns=lst.count('NOUN')
    Num=lst.count('NUM')
    Part=lst.count('PART')
    Pronoun=lst.count('PRON')
    Punct=lst.count('PUNCT')
    Verb=lst.count('VERB')
    X=[Adjs,Advs,Aux,Conj,Det,Inter,Nouns,Num,Part,Pronoun,Punct,Verb]
    return X


In [13]:
df['POS']=df['text'].apply(lambda x:PosFeatures(x))
df.loc[:, 'Adjs'] = df.POS.map(lambda x: x[0])
df.loc[:, 'Advs'] = df.POS.map(lambda x: x[1])
df.loc[:, 'Aux'] = df.POS.map(lambda x: x[2])
df.loc[:, 'Conj'] = df.POS.map(lambda x: x[3])
df.loc[:, 'Det'] = df.POS.map(lambda x: x[4])
df.loc[:, 'Inter'] = df.POS.map(lambda x: x[5])
df.loc[:, 'Noun'] = df.POS.map(lambda x: x[6])
df.loc[:, 'Numeric'] = df.POS.map(lambda x: x[7])
df.loc[:, 'Part'] = df.POS.map(lambda x: x[8])
df.loc[:, 'Pronoun'] = df.POS.map(lambda x: x[9])
df.loc[:, 'Punct'] = df.POS.map(lambda x: x[10])
df.loc[:, 'Verb'] = df.POS.map(lambda x: x[11])



In [14]:
def Lemmas(s):
   return ' '.join(str(c.lemma_) for c in s)



In [15]:
df['textlemma']=df['text'].apply(lambda x: Lemmas(x))

In [16]:
df

,_id,topic,text,title,year,month,day,key,url,dropkey,...,Conj,Det,Inter,Noun,Numeric,Part,Pronoun,Punct,Verb,textlemma
33977,{'$oid': '611d9fe6049812a18b162994'},politica,"(el, presidente, ivan, duque, llamo, ayer, al,...",¿De qué hablaron Duque y Uribe en llamada tras...,2020,8,5,politica-gobierno-presidente-ivan-duque-se-pro...,http://www.eltiempo.com//politica/gobierno/pre...,El presidente Iván Duque llamó ayer al expresi...,...,0,53,0,90,1,1,23,52,48,el presidente ivan duque llamar ayer al expres...
10983,{'$oid': '611d9fe3049812a18b146d35'},cultura,"(“, no, me, afecta, ,, la, gente, puede, decir...",Hijo de Gloria Trevi habla sobre los ataques a...,2021,4,30,cultura-musica-y-libros-hijo-de-gloria-trevi-s...,http://www.eltiempo.com//cultura/musica-y-libr...,"“No me afecta, la gente puede decir lo que qui...",...,0,32,0,58,4,0,25,33,35,"“ no yo afectar , el gente poder decir él que ..."
38588,{'$oid': '611d9fe7049812a18b16aaa9'},tecnosfera,"(el, consumer, electronic, show, (, ces, ), de...","Televisores inteligentes, 5G y las novedades d...",2020,1,8,tecnosfera-novedades-tecnologia-novedades-que-...,http://www.eltiempo.com//tecnosfera/novedades-...,El Consumer Electronic Show (CES) de Las Vegas...,...,0,99,1,193,24,0,22,74,39,el consumer electronic show ( ces ) de el vega...
43605,{'$oid': '611d9fe8049812a18b1787ba'},vida,"(en, uno, de, los, momentos, de, la, historia,...",Qué dice la neurobiología de la distancia soci...,2020,12,12,vida-que-dice-la-neurobiologia-de-la-distancia...,http://www.eltiempo.com//vida/que-dice-la-neur...,En uno de los momentos de la historia en que m...,...,0,225,0,327,14,0,96,200,190,en uno de el momento de el historia en que mas...
31021,{'$oid': '611d9fe5049812a18b15e51c'},economia,"(segun, un, nuevo, estudio, de, oracle, y, wor...",La pandemia hizo del 2020 el año laboral más e...,2020,10,12,economia-sectores-coranavirus-el-2020-es-el-an...,http://www.eltiempo.com//economia/sectores/cor...,Según un nuevo estudio de Oracle y Workplace I...,...,0,88,0,146,35,0,12,54,38,segun uno nuevo estudio de oracle y workplace ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4982,{'$oid': '611d9fe2049812a18b1403ac'},cultura,"(este, 20, de, agosto, se, conocio, el, partic...",Video: el curioso protocolo de bioseguridad de...,2020,8,21,cultura-gente-barranquilla-particular-protocol...,http://www.eltiempo.com//cultura/gente/barranq...,Este 20 de agosto se conoció el particular vid...,...,0,55,0,71,9,0,22,19,37,este 20 de agosto él conocio el particular vid...
8912,{'$oid': '611d9fe2049812a18b144842'},cultura,"(pocos, desafios, mas, complejos, para, un, es...",Así mataron a papá: Jorge Emilio Sierra,2020,3,28,cultura-musica-y-libros-asi-mataron-a-papa-fra...,http://www.eltiempo.com//cultura/musica-y-libr...,Pocos desafíos más complejos para un escritor ...,...,0,266,0,361,29,0,146,263,199,poco desafio mas complejo para uno escritor qu...
36116,{'$oid': '611d9fe6049812a18b165728'},politica,"(la, modalidad, de, contrato, por, horas, fue,...",¿El Congreso abrió la puerta al contrato de tr...,2020,8,26,politica-congreso-proyecto-que-estableceria-el...,http://www.eltiempo.com//politica/congreso/pro...,La modalidad de contrato por horas fue uno de ...,...,0,102,0,154,7,2,43,74,60,el modalidad de contrato por hora ser uno de e...
40827,{'$oid': '611d9fe8049812a18b17327d'},vida,"(', soy, de, la, generacion, igualdad, ’, ,, d...",El empoderamiento de la mujer cuestiona reinad...,2020,3,14,vida-mujeres-el-empoderamiento-de-la-mujer-cue...,http://www.eltiempo.com//vida/mujeres/el-empod...,"'Soy de la generación igualdad’, dice el lema ...",...,0,258,0,381,20,0,86,174,164,"' ser de el generacion igualdad ’ , decir el l..."


In [17]:
df_final=df[['dropkey','Adjs', 'Advs', 'Aux', 'Conj', 'Det', 'Inter', 'Noun',
       'Numeric', 'Part', 'Pronoun', 'Punct', 'Verb', 'textlemma']]

In [ ]:
#df_final.to_csv('/content/drive/MyDrive/Proyecto1PLN/POS_60K_PARTE6.csv')

### Union Dfs

In [20]:
df_p1=pd.read_csv('POS_10K_PARTE1.csv', 'utf-8',delimiter=',',index_col=0)
df_p2=pd.read_csv('POS_20K_PARTE2.csv', 'utf-8',delimiter=',',index_col=0)
df_p3=pd.read_csv('POS_30K_PARTE3.csv', 'utf-8',delimiter=',',index_col=0)
df_p4=pd.read_csv('POS_40K_PARTE4.csv', 'utf-8',delimiter=',',index_col=0)
df_p5=pd.read_csv('POS_50K_PARTE5.csv', 'utf-8',delimiter=',',index_col=0)

In [21]:
df_union=pd.concat([df_p1,df_p2,df_p3,df_p4,df_p5])
df_union

,dropkey,Adjs,Advs,Aux,Conj,Det,Inter,Noun,Numeric,Part,Pronoun,Punct,Verb,textlemma
0,cultura-musica-y-libros-vallenatos-de-el-trio-...,1,0,0,0,3,0,3,0,0,0,0,0,el vallenato con guitarra de el hijo de ' el c...
1,cultura-gastronomia-entrevista-con-consol-rodr...,1,0,0,0,3,0,2,0,0,0,0,0,el secreto de uno guru de el comida crudivegán
2,OBJ24062586_13 by Yhonatan Loaiza on ScribdEl...,31,8,13,0,42,0,84,6,1,15,67,21,obj24062586_13 by yhonatan loaiza on scribde...
3,"""'Origen' me regresó a la infancia para record...",7,6,6,0,29,0,42,0,0,12,25,19,""" ' origen ' yo regreso a el infancia para rec..."
4,"""A Slow Fire Burning"" es el título de la nueva...",21,2,9,0,41,0,62,4,0,18,49,41,""" a slow fire burning "" ser el titulo de el nu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47566,“Él no es así; él nunca se comportó de esa man...,93,39,53,0,169,0,257,10,0,73,152,140,“ el no ser asi ; el nunca él comportar de ese...
47567,“Únete a nosotros este jueves 18 de febrero pa...,41,5,9,0,77,0,111,10,0,22,46,39,“ unete a yo este jueves 18 de febrero para ex...
47568,“Únete a nosotros este jueves 18 de febrero pa...,41,5,9,0,77,0,111,10,0,22,46,39,“ unete a yo este jueves 18 de febrero para ex...
47569,"“‘Hoy tengo que mostrar la Luna’, decía, preoc...",53,4,9,0,52,0,111,7,0,8,51,21,"“ ‘ hoy tener que mostrar el luna ’ , decir , ..."


In [22]:
df_consolidado=pd.merge(datasetMaterial,df_union,on='dropkey')
df_consolidado

,_id,topic,text,title,year,month,day,key,url,dropkey,...,Conj,Det,Inter,Noun,Numeric,Part,Pronoun,Punct,Verb,textlemma
0,{'$oid': '611d9fe1049812a18b13a8dd'},cultura,el vallenato con guitarra de los hijos de 'la ...,El vallenato con guitarra de los hijos de 'La ...,2020,12,7,cultura-musica-y-libros-vallenatos-de-el-trio-...,http://www.eltiempo.com//cultura/musica-y-libr...,cultura-musica-y-libros-vallenatos-de-el-trio-...,...,0,3,0,3,0,0,0,0,0,el vallenato con guitarra de el hijo de ' el c...
1,{'$oid': '611d9fe1049812a18b13a8de'},cultura,los secretos de una guru de la comida crudivegana,Los secretos de una gurú de la comida crudivegana,2021,4,10,cultura-gastronomia-entrevista-con-consol-rodr...,http://www.eltiempo.com//cultura/gastronomia/e...,cultura-gastronomia-entrevista-con-consol-rodr...,...,0,3,0,2,0,0,0,0,0,el secreto de uno guru de el comida crudivegán
2,{'$oid': '611d9fe1049812a18b13a8df'},cultura,obj24062586_13 by yhonatan loaiza on scribdel...,Teatro colombiano que conquista el mundo,2020,9,30,cultura-arte-y-teatro-obras-de-teatro-colombia...,http://www.eltiempo.com//cultura/arte-y-teatro...,OBJ24062586_13 by Yhonatan Loaiza on ScribdEl...,...,0,42,0,84,6,1,15,67,21,obj24062586_13 by yhonatan loaiza on scribde...
3,{'$oid': '611d9fe1049812a18b13a8e0'},cultura,"""'origen' me regreso a la infancia para record...",Juanes presenta el video de 'Sin medir distancia',2021,7,8,cultura-musica-y-libros-juanes-presenta-el-vid...,http://www.eltiempo.com//cultura/musica-y-libr...,"""'Origen' me regresó a la infancia para record...",...,0,29,0,42,0,0,12,25,19,""" ' origen ' yo regreso a el infancia para rec..."
4,{'$oid': '611d9fe1049812a18b13a8e5'},cultura,"""a slow fire burning"" es el titulo de la nueva...",Autora de 'La chica del tren' anuncia su nueva...,2020,12,3,cultura-musica-y-libros-la-chica-del-tren-paul...,http://www.eltiempo.com//cultura/musica-y-libr...,"""A Slow Fire Burning"" es el título de la nueva...",...,0,41,0,62,4,0,18,49,41,""" a slow fire burning "" ser el titulo de el nu..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47566,{'$oid': '611d9fe9049812a18b180125'},vida,“el no es asi; el nunca se comporto de esa man...,Consejos para reconocer una relación tóxica y ...,2021,3,4,vida-familia-parejas-como-reconocer-una-relaci...,http://www.eltiempo.com//vida/familia/parejas-...,“Él no es así; él nunca se comportó de esa man...,...,0,169,0,257,10,0,73,152,140,“ el no ser asi ; el nunca él comportar de ese...
47567,{'$oid': '611d9fe9049812a18b18012b'},vida,“unete a nosotros este jueves 18 de febrero pa...,La Nasa retransmitirá en español el amartizaje...,2021,2,19,vida-ciencia-la-nasa-retransmite-en-espanol-el...,http://www.eltiempo.com//vida/ciencia/la-nasa-...,“Únete a nosotros este jueves 18 de febrero pa...,...,0,77,0,111,10,0,22,46,39,“ unete a yo este jueves 18 de febrero para ex...
47568,{'$oid': '611d9fe9049812a18b18012c'},vida,“unete a nosotros este jueves 18 de febrero pa...,La Nasa retransmitirá en español el amartizaje...,2021,2,19,vida-ciencia-la-nasa-retransmite-en-espanol-el...,http://www.eltiempo.com//vida/ciencia/la-nasa-...,“Únete a nosotros este jueves 18 de febrero pa...,...,0,77,0,111,10,0,22,46,39,“ unete a yo este jueves 18 de febrero para ex...
47569,{'$oid': '611d9fe9049812a18b18012e'},vida,"“‘hoy tengo que mostrar la luna’, decia, preoc...",Cien años sin el genio de Julio Garavito Armero,2020,3,10,vida-ciencia-cien-anos-sin-el-genio-de-julio-g...,http://www.eltiempo.com//vida/ciencia/cien-ano...,"“‘Hoy tengo que mostrar la Luna’, decía, preoc...",...,0,52,0,111,7,0,8,51,21,"“ ‘ hoy tener que mostrar el luna ’ , decir , ..."


### Explorar

In [23]:
df.topic.value_counts(normalize=True)

deportes      0.298024
cultura       0.237495
politica      0.143338
vida          0.138714
economia      0.124422
tecnosfera    0.058008
Name: topic, dtype: float64

In [24]:
def Root(s):
   return [c.lemma_ for c in s if c.dep_ =='ROOT' and c.is_stop ==0 and c.is_punct==0]


In [25]:
df['textlemma']=df['text'].apply(lambda x: Lemmas(x))
df['Root']=df['text'].apply(lambda x: Root(x))

In [26]:
df.head(50)

,_id,topic,text,title,year,month,day,key,url,dropkey,...,Det,Inter,Noun,Numeric,Part,Pronoun,Punct,Verb,textlemma,Root
33977,{'$oid': '611d9fe6049812a18b162994'},politica,"(el, presidente, ivan, duque, llamo, ayer, al,...",¿De qué hablaron Duque y Uribe en llamada tras...,2020,8,5,politica-gobierno-presidente-ivan-duque-se-pro...,http://www.eltiempo.com//politica/gobierno/pre...,El presidente Iván Duque llamó ayer al expresi...,...,53,0,90,1,1,23,52,48,el presidente ivan duque llamar ayer al expres...,"[llamar, enfrentar, interesar, duque, , esperar]"
10983,{'$oid': '611d9fe3049812a18b146d35'},cultura,"(“, no, me, afecta, ,, la, gente, puede, decir...",Hijo de Gloria Trevi habla sobre los ataques a...,2021,4,30,cultura-musica-y-libros-hijo-de-gloria-trevi-s...,http://www.eltiempo.com//cultura/musica-y-libr...,"“No me afecta, la gente puede decir lo que qui...",...,32,0,58,4,0,25,33,35,"“ no yo afectar , el gente poder decir él que ...","[cambiar, prision]"
38588,{'$oid': '611d9fe7049812a18b16aaa9'},tecnosfera,"(el, consumer, electronic, show, (, ces, ), de...","Televisores inteligentes, 5G y las novedades d...",2020,1,8,tecnosfera-novedades-tecnologia-novedades-que-...,http://www.eltiempo.com//tecnosfera/novedades-...,El Consumer Electronic Show (CES) de Las Vegas...,...,99,1,193,24,0,22,74,39,el consumer electronic show ( ces ) de el vega...,"[comenzar, exhibición, uu, presentar, esperar,..."
43605,{'$oid': '611d9fe8049812a18b1787ba'},vida,"(en, uno, de, los, momentos, de, la, historia,...",Qué dice la neurobiología de la distancia soci...,2020,12,12,vida-que-dice-la-neurobiologia-de-la-distancia...,http://www.eltiempo.com//vida/que-dice-la-neur...,En uno de los momentos de la historia en que m...,...,225,0,327,14,0,96,200,190,en uno de el momento de el historia en que mas...,"[quedar, asegurar, convencido, recalca, estima..."
31021,{'$oid': '611d9fe5049812a18b15e51c'},economia,"(segun, un, nuevo, estudio, de, oracle, y, wor...",La pandemia hizo del 2020 el año laboral más e...,2020,10,12,economia-sectores-coranavirus-el-2020-es-el-an...,http://www.eltiempo.com//economia/sectores/cor...,Según un nuevo estudio de Oracle y Workplace I...,...,88,0,146,35,0,12,54,38,segun uno nuevo estudio de oracle y workplace ...,"[preferiria, acabar]"
30859,{'$oid': '611d9fe5049812a18b15e133'},economia,"(por, haber, impuesto, un, termino, distinto, ...",Supertransporte multa a Viva Air por vulnerar ...,2021,1,23,economia-empresas-viva-air-por-que-la-supertra...,http://www.eltiempo.com//economia/empresas/viv...,Por haber impuesto un término distinto del pre...,...,140,0,207,8,1,48,67,83,por haber imponer uno termino distinto del pre...,"[imponer, explicar, agregar, proferir, fortale..."
41371,{'$oid': '611d9fe8049812a18b1741f6'},vida,"(china, planea, construir, un, prototipo, para...","Una estación orbital, nuevo objetivo de China ...",2020,11,26,vida-ciencia-una-estacion-orbital-nuevo-objeti...,http://www.eltiempo.com//vida/ciencia/una-esta...,China planea construir un prototipo para una e...,...,35,0,51,0,0,2,20,20,china planear construir uno prototipo para uno...,"[ , agregar, cientifico.madrid]"
42285,{'$oid': '611d9fe8049812a18b175e6e'},vida,"(el, informe, planeta, vivo, 2020, de, la, wwf...",La vacuna para cuidar la biodiversidad sí exis...,2020,9,15,vida-medio-ambiente-la-vacuna-para-cuidar-la-b...,http://www.eltiempo.com//vida/medio-ambiente/l...,El Informe Planeta Vivo 2020 de la WWF y la So...,...,70,0,103,21,0,21,54,32,el informe planeta vivo 2020 de el wwf y el so...,"[desalentador, reportar, venir, enfermedad]"
8942,{'$oid': '611d9fe2049812a18b1448d7'},cultura,"(por, fin, se, develo, el, misterio, sobre, cu...",Laura Acuña llega a programa matutino de Carac...,2021,5,22,cultura-cine-y-tv-presentadora-laura-acuna-lle...,http://www.eltiempo.com//cultura/cine-y-tv/pre...,Por fin se develó el misterio sobre cuál sería...,...,44,0,62,8,2,14,28,27,por fin él develo el misterio sobre cual seria...,"[develo, confirmar, , venir]"
32931,{'$o

In [27]:
def ner(s):
   return [{c.text:c.label_}for c in s.ents ]

In [28]:
df['NER']=df['text'].apply(lambda x: ner(x))

In [29]:
x=list(df['NER'][1]) 
x

KeyError: 1

In [30]:
y=df['text'].head(1)
y.values

array([el presidente ivan duque llamo ayer al expresidente alvaro uribe velez, a quien la corte suprema de justicia le ordeno la detencion domiciliaria, y revelo que lo encontro con la frente en alto y con la certeza de que se defendera de manera gallarda.el jefe de estado defendio la “inocencia y honorabilidad” del hoy senador alvaro uribe y pidio que “una persona que combatio el terrorismo y la criminalidad” se pueda defender en libertad. “alvaro uribe ha enfrentado todos y cada uno de los cuestionamientos con estoicismo. una persona que ha tenido el honor de presidir este pais debe poder defenderse en libertad”, manifesto duque este miercoles, en entrevista con la fm.(tambien le puede interesar: el pais, sacudido tras la detencion de alvaro uribe)cabe recordar que la investigacion contra uribe se inicio en julio de 2018, luego de que el entonces magistrado jose luis barcelo decidio llamarlo a indagatoria al encontrar que, al parecer, uribe estaba intentando torcer el testimonio de t

### Vectorización sobre TextLemma e incluyendo POS

In [31]:
df.head()

,_id,topic,text,title,year,month,day,key,url,dropkey,...,Inter,Noun,Numeric,Part,Pronoun,Punct,Verb,textlemma,Root,NER
33977,{'$oid': '611d9fe6049812a18b162994'},politica,"(el, presidente, ivan, duque, llamo, ayer, al,...",¿De qué hablaron Duque y Uribe en llamada tras...,2020,8,5,politica-gobierno-presidente-ivan-duque-se-pro...,http://www.eltiempo.com//politica/gobierno/pre...,El presidente Iván Duque llamó ayer al expresi...,...,0,90,1,1,23,52,48,el presidente ivan duque llamar ayer al expres...,"[llamar, enfrentar, interesar, duque, , esperar]","[{'ivan duque': 'PER'}, {'alvaro uribe velez':..."
10983,{'$oid': '611d9fe3049812a18b146d35'},cultura,"(“, no, me, afecta, ,, la, gente, puede, decir...",Hijo de Gloria Trevi habla sobre los ataques a...,2021,4,30,cultura-musica-y-libros-hijo-de-gloria-trevi-s...,http://www.eltiempo.com//cultura/musica-y-libr...,"“No me afecta, la gente puede decir lo que qui...",...,0,58,4,0,25,33,35,"“ no yo afectar , el gente poder decir él que ...","[cambiar, prision]","[{'angel gabriel': 'PER'}, {'gloria trevi': 'P..."
38588,{'$oid': '611d9fe7049812a18b16aaa9'},tecnosfera,"(el, consumer, electronic, show, (, ces, ), de...","Televisores inteligentes, 5G y las novedades d...",2020,1,8,tecnosfera-novedades-tecnologia-novedades-que-...,http://www.eltiempo.com//tecnosfera/novedades-...,El Consumer Electronic Show (CES) de Las Vegas...,...,1,193,24,0,22,74,39,el consumer electronic show ( ces ) de el vega...,"[comenzar, exhibición, uu, presentar, esperar,...","[{'las vegas': 'LOC'}, {'nevada': 'LOC'}, {'ee..."
43605,{'$oid': '611d9fe8049812a18b1787ba'},vida,"(en, uno, de, los, momentos, de, la, historia,...",Qué dice la neurobiología de la distancia soci...,2020,12,12,vida-que-dice-la-neurobiologia-de-la-distancia...,http://www.eltiempo.com//vida/que-dice-la-neur...,En uno de los momentos de la historia en que m...,...,0,327,14,0,96,200,190,en uno de el momento de el historia en que mas...,"[quedar, asegurar, convencido, recalca, estima...","[{'organizacion mundial de la salud': 'ORG'}, ..."
31021,{'$oid': '611d9fe5049812a18b15e51c'},economia,"(segun, un, nuevo, estudio, de, oracle, y, wor...",La pandemia hizo del 2020 el año laboral más e...,2020,10,12,economia-sectores-coranavirus-el-2020-es-el-an...,http://www.eltiempo.com//economia/sectores/cor...,Según un nuevo estudio de Oracle y Workplace I...,...,0,146,35,0,12,54,38,segun uno nuevo estudio de oracle y workplace ...,"[preferiria, acabar]","[{'oracle': 'MISC'}, {'canada': 'LOC'}, {'cien..."


In [34]:
df.shape

(2379, 26)

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from spacy.lang.es.stop_words import STOP_WORDS

In [35]:
vectorizador1 = CountVectorizer(stop_words = STOP_WORDS, min_df = 5, analyzer = "word", )
matrizCountVectorizer1 = vectorizador1.fit_transform(df['textlemma'])
type(matrizCountVectorizer1)

scipy.sparse.csr.csr_matrix

In [41]:
vectorizador4 = TfidfVectorizer(max_df=0.9, min_df = 2, max_features = 20000, lowercase = True, stop_words=STOP_WORDS, use_idf = True)
matrizTFIDF = vectorizador4.fit_transform(df['textlemma'])

# Para visualizarlo lo convertiremos en dataframe 
count_array = matrizTFIDF.toarray()
dfCount = pd.DataFrame(data = count_array, columns = vectorizador4.get_feature_names())
final = dfCount.transpose()
final.head()


,0,1,2,3,4,5,6,7,8,9,...,2369,2370,2371,2372,2373,2374,2375,2376,2377,2378
00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000en,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(matrizTFIDF,
                                               df[['Noun','Inter','Numeric', 'Part','Pronoun','Punct','topic']],
                                                test_size=0.3,
                                                random_state=1,
                                                stratify = df['topic'])

In [73]:
train_X.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.26454817, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [86]:
train_X.shape

(1665, 20000)

In [81]:
np.array(train_y.iloc[:,0:6])

array([[ 45,   0,   1,   0,   5,  17],
       [130,   0,  19,   0,  35,  66],
       [ 33,   0,  13,   0,   5,  11],
       ...,
       [100,   1,   6,   0,  21,  51],
       [ 46,   1,   0,   0,  20,  33],
       [168,   0,  19,   0,  23,  89]], dtype=int64)

In [87]:
X_train = np.column_stack((train_X.toarray(),np.array(train_y.iloc[:,0:6])))

In [88]:
X_train.shape

(1665, 20006)

In [91]:
 train_y.drop(['Noun','Inter','Numeric', 'Part','Pronoun','Punct'], inplace=True, axis=1)

In [95]:
train_y.head()

,topic
25140,deportes
44842,vida
14430,deportes
24221,deportes
6398,cultura


In [100]:
X_test = np.column_stack((test_X.toarray(),np.array(test_y.iloc[:,0:6])))

In [101]:
test_y.drop(['Noun','Inter','Numeric', 'Part','Pronoun','Punct'], inplace=True, axis=1)

In [99]:
clf = MultinomialNB().fit(X_train, train_y)

In [105]:
predicted = clf.predict(X_test)
print(metrics.classification_report(test_y,predicted))

              precision    recall  f1-score   support

     cultura       0.28      1.00      0.44       170
    deportes       0.91      0.45      0.60       213
    economia       0.80      0.09      0.16        89
    politica       0.00      0.00      0.00       102
  tecnosfera       0.00      0.00      0.00        41
        vida       1.00      0.01      0.02        99

    accuracy                           0.38       714
   macro avg       0.50      0.26      0.20       714
weighted avg       0.58      0.38      0.31       714



In [106]:
clfRf = RandomForestClassifier(max_depth = 8, 
                             random_state = 0,
                             n_estimators = 200, 
                             min_samples_split = 5,
                             class_weight = 'balanced',
                             criterion = 'entropy')

In [107]:
clfRf.fit(X_train, train_y)
predictedRf = clfRf.predict(X_test)
print(metrics.classification_report(test_y,predictedRf))

              precision    recall  f1-score   support

     cultura       0.85      0.82      0.83       170
    deportes       0.91      0.97      0.94       213
    economia       0.77      0.91      0.84        89
    politica       0.90      0.88      0.89       102
  tecnosfera       0.85      0.83      0.84        41
        vida       0.74      0.59      0.66        99

    accuracy                           0.85       714
   macro avg       0.84      0.83      0.83       714
weighted avg       0.85      0.85      0.85       714



In [108]:
predictedRf

array(['cultura', 'cultura', 'politica', 'deportes', 'economia', 'vida',
       'cultura', 'deportes', 'economia', 'deportes', 'politica',
       'economia', 'politica', 'politica', 'cultura', 'economia',
       'economia', 'deportes', 'deportes', 'deportes', 'tecnosfera',
       'economia', 'politica', 'tecnosfera', 'politica', 'deportes',
       'deportes', 'tecnosfera', 'cultura', 'cultura', 'cultura',
       'deportes', 'politica', 'economia', 'deportes', 'economia',
       'deportes', 'deportes', 'deportes', 'vida', 'vida', 'cultura',
       'deportes', 'deportes', 'deportes', 'deportes', 'deportes',
       'deportes', 'deportes', 'economia', 'deportes', 'cultura',
       'cultura', 'cultura', 'vida', 'economia', 'deportes', 'deportes',
       'deportes', 'economia', 'deportes', 'deportes', 'deportes',
       'cultura', 'vida', 'politica', 'economia', 'cultura', 'tecnosfera',
       'economia', 'economia', 'deportes', 'cultura', 'deportes',
       'politica', 'economia', 'politica

### Red Neuronal

In [109]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [131]:
def modelo_red(X, opt = 'adam', act = 'sigmoid'):
    
    """
    Retorna una arquitectura
    de red
    """
    #layers.Dropout(0.5), #Ponerle ruido a la red
    model = keras.Sequential([layers.Dense(64, activation=act, input_shape=[X.shape[1]]),
                              layers.Dense(64,activation = act),
                              layers.Dense(64,activation = act),
                              layers.Dense(6,activation = 'softmax')])
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [156]:
#print(X_train.dtypes)
type(train_y)

pandas.core.frame.DataFrame

In [132]:
modelo = modelo_red(X_train,act = 'relu')

In [133]:
modelo.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 64)                1280448   
_________________________________________________________________
dense_13 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_14 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_15 (Dense)             (None, 6)                 390       
Total params: 1,289,158
Trainable params: 1,289,158
Non-trainable params: 0
_________________________________________________________________


In [172]:
def Escalar_topicos(x):
    if x == 'deportes':
        return 0
    elif x== 'vida':
        return 1
    elif x=='cultura':
        return 2
    elif x=='economia':
        return 3
    elif x=='politica':
        return 4
    else:
        return 5
#Escalar_topicos('vida')   
y_train = train_y.topic.apply(Escalar_topicos)
#

In [179]:
y_test = test_y.topic.apply(Escalar_topicos)

In [180]:
from tensorflow.keras.callbacks import EarlyStopping
bl=0.7
# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss',
                               verbose=1, 
                               patience=10, 
                               baseline = bl,
                               restore_best_weights = True)

In [181]:
entrenamiento = modelo.fit(X_train,
                           np.array(y_train),
                           epochs=1000,
                           validation_data = (X_test, y_test),
                           batch_size = 16,
                           callbacks=[early_stopping]
                          )

Epoch 1/1000
105/105 [==============================] - 1s 12ms/step - loss: 2.8844e-05 - accuracy: 1.0000 - val_loss: 0.6434 - val_accuracy: 0.8880
Epoch 2/1000
105/105 [==============================] - 1s 8ms/step - loss: 2.5921e-05 - accuracy: 1.0000 - val_loss: 0.6472 - val_accuracy: 0.8880
Epoch 3/1000
105/105 [==============================] - 1s 7ms/step - loss: 2.3393e-05 - accuracy: 1.0000 - val_loss: 0.6505 - val_accuracy: 0.8880
Epoch 4/1000
105/105 [==============================] - 1s 8ms/step - loss: 2.1212e-05 - accuracy: 1.0000 - val_loss: 0.6540 - val_accuracy: 0.8880
Epoch 5/1000
105/105 [==============================] - 1s 7ms/step - loss: 1.9353e-05 - accuracy: 1.0000 - val_loss: 0.6568 - val_accuracy: 0.8880
Epoch 6/1000
105/105 [==============================] - 1s 7ms/step - loss: 1.7654e-05 - accuracy: 1.0000 - val_loss: 0.6599 - val_accuracy: 0.8866
Epoch 7/1000
105/105 [==============================] - 1s 8ms/step - loss: 1.6276e-05 - accuracy: 1.0000 - val

In [182]:
def DesEscalar_topicos(x):
    if x == 0:
        return 'deportes'
    elif x== 1 :
        return 'vida'
    elif x== 2:
        return 'cultura'
    elif x== 3:
        return 'economia'
    elif x== 4:
        return 'politica'
    else:
        return 'tecnosfera'

In [183]:
prediccionRed = modelo.predict(X_test)

In [203]:
print(DesEscalar_topicos(np.argmax(prediccionRed[345])))
print(test_y.iloc[345,0])

tecnosfera
tecnosfera


In [205]:
print(len(test_y))
print(prediccionRed.shape)

714
(714, 6)


In [38]:
vectorizador5 = TfidfVectorizer(max_df=0.8, min_df=2, max_features=20000, lowercase = True, stop_words=STOP_WORDS, use_idf = True, ngram_range=(2,5),strip_accents='unicode')
matrizTFIDFnGrams = vectorizador5.fit_transform(df['text_lower'])
# Para visualizarlo lo convertiremos en dataframe 
count_array = matrizTFIDFnGrams.toarray()
dfCount = pd.DataFrame(data = count_array, columns = vectorizador5.get_feature_names())
final = dfCount.transpose()
final.head(30)

KeyError: 'text_lower'

In [39]:
# Observa que al cambiar el parametro ngram_range también podemos observar el TF-IDF para la unión de 2 y 3 palabras que aparecen juntas en los textos.
vectorizador5 = TfidfVectorizer(max_df=0.8, min_df=2, max_features=20000, lowercase = True, stop_words=STOP_WORDS, use_idf = True, ngram_range=(2,5),strip_accents='unicode')
matrizTFIDFnGrams = vectorizador5.fit_transform(df['text_lower'])
# Para visualizarlo lo convertiremos en dataframe 
count_array = matrizTFIDFnGrams.toarray()
dfCount = pd.DataFrame(data = count_array, columns = vectorizador5.get_feature_names())
final = dfCount.transpose()
final.head(30)

KeyError: 'text_lower'